In [1]:
"""
Create and operate on grids and profiles.

**Grid generation**

* :func:`~fatiando.gridder.regular`
* :func:`~fatiando.gridder.scatter`

**Grid operations**

* :func:`~fatiando.gridder.cut`
* :func:`~fatiando.gridder.profile`

**Interpolation**

* :func:`~fatiando.gridder.interp`
* :func:`~fatiando.gridder.interp_at`
* :func:`~fatiando.gridder.extrapolate_nans`

**Input/Output**

* :func:`~fatiando.gridder.load_surfer`: Read a Surfer grid file and return
  three 1d numpy arrays and the grid shape

**Misc**

* :func:`~fatiando.gridder.spacing`

----

"""

import numpy
import scipy.interpolate
import matplotlib.mlab


def load_surfer(fname, fmt='ascii'):
    """
    Read a Surfer grid file and return three 1d numpy arrays and the grid shape

    Surfer is a contouring, gridding and surface mapping software
    from GoldenSoftware. The names and logos for Surfer and Golden
    Software are registered trademarks of Golden Software, Inc.

    http://www.goldensoftware.com/products/surfer

    According to Surfer structure, x and y are horizontal and vertical
    screen-based coordinates respectively. If the grid is in geographic
    coordinates, x will be longitude and y latitude. If the coordinates
    are cartesian, x will be the easting and y the norting coordinates.

    WARNING: This is opposite to the convention used for Fatiando.
    See io_surfer.py in cookbook.

    Parameters:

    * fname : str
        Name of the Surfer grid file
    * fmt : str
        File type, can be 'ascii' or 'binary'

    Returns:

    * x : 1d-array
        Value of the horizontal coordinate of each grid point.
    * y : 1d-array
        Value of the vertical coordinate of each grid point.
    * grd : 1d-array
        Values of the field in each grid point. Field can be for example
        topography, gravity anomaly etc
    * shape : tuple = (ny, nx)
        The number of points in the vertical and horizontal grid dimensions,
        respectively

    """
    assert fmt in ['ascii', 'binary'], "Invalid grid format '%s'. Should be \
        'ascii' or 'binary'." % (fmt)
    if fmt == 'ascii':
        # Surfer ASCII grid structure
        # DSAA            Surfer ASCII GRD ID
        # nCols nRows     number of columns and rows
        # xMin xMax       X min max
        # yMin yMax       Y min max
        # zMin zMax       Z min max
        # z11 z21 z31 ... List of Z values
        with open(fname) as ftext:
            # DSAA is a Surfer ASCII GRD ID
            id = ftext.readline()
            # Read the number of columns (nx) and rows (ny)
            nx, ny = [int(s) for s in ftext.readline().split()]
            # Read the min/max value of x (columns/longitue)
            xmin, xmax = [float(s) for s in ftext.readline().split()]
            # Read the min/max value of  y(rows/latitude)
            ymin, ymax = [float(s) for s in ftext.readline().split()]
            # Read the min/max value of grd
            zmin, zmax = [float(s) for s in ftext.readline().split()]
            data = numpy.fromiter((float(i) for line in ftext for i in
                                   line.split()), dtype='f')
            grd = numpy.ma.masked_greater_equal(data, 1.70141e+38)
        # Create x and y numpy arrays
        x = numpy.linspace(xmin, xmax, nx)
        y = numpy.linspace(ymin, ymax, ny)
        x, y = [tmp.ravel() for tmp in numpy.meshgrid(x, y)]
    if fmt == 'binary':
        raise NotImplementedError(
            "Binary file support is not implemented yet.")
    return x, y, grd, (ny, nx)

def regular(area, shape, z=None):
    """
    Create a regular grid. Order of the output grid is x varies first, then y.

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * shape
        Shape of the regular grid, ie ``(ny, nx)``.
    * z
        Optional. z coordinate of the grid points. If given, will return an
        array with the value *z*.

    Returns:

    * ``[xcoords, ycoords]``
        Numpy arrays with the x and y coordinates of the grid points
    * ``[xcoords, ycoords, zcoords]``
        If *z* given. Numpy arrays with the x, y, and z coordinates of the grid
        points

    """
    ny, nx = shape
    x1, x2, y1, y2 = area
    dy, dx = spacing(area, shape)
    x_range = numpy.arange(x1, x2, dx)
    y_range = numpy.arange(y1, y2, dy)
    # Need to make sure that the number of points in the grid is correct
    # because of rounding errors in arange. Sometimes x2 and y2 are included,
    # sometimes not
    if len(x_range) < nx:
        x_range = numpy.append(x_range, x2)
    if len(y_range) < ny:
        y_range = numpy.append(y_range, y2)
    assert len(x_range) == nx, "Failed! x_range doesn't have nx points"
    assert len(y_range) == ny, "Failed! y_range doesn't have ny points"
    xcoords, ycoords = [mat.ravel()
                        for mat in numpy.meshgrid(x_range, y_range)]
    if z is not None:
        zcoords = z * numpy.ones_like(xcoords)
        return [xcoords, ycoords, zcoords]
    else:
        return [xcoords, ycoords]

def scatter(area, n, z=None, seed=None):
    """
    Create an irregular grid with a random scattering of points.

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * n
        Number of points
    * z
        Optional. z coordinate of the points. If given, will return an
        array with the value *z*.
    * seed : None or int
        Seed used to generate the pseudo-random numbers. If `None`, will use a
        different seed every time. Use the same seed to generate the same
        random points.

    Returns:

    * ``[xcoords, ycoords]``
        Numpy arrays with the x and y coordinates of the points
    * ``[xcoords, ycoords, zcoords]``
        If *z* given. Arrays with the x, y, and z coordinates of the points

    """
    x1, x2, y1, y2 = area
    numpy.random.seed(seed)
    xcoords = numpy.random.uniform(x1, x2, n)
    ycoords = numpy.random.uniform(y1, y2, n)
    numpy.random.seed()
    if z is not None:
        zcoords = z * numpy.ones(n)
        return [xcoords, ycoords, zcoords]
    else:
        return [xcoords, ycoords]

def spacing(area, shape):
    """
    Returns the spacing between grid nodes

    Parameters:

    * area
        ``(x1, x2, y1, y2)``: Borders of the grid
    * shape
        Shape of the regular grid, ie ``(ny, nx)``.

    Returns:

    * ``[dy, dx]``
        Spacing the y and x directions

    """
    x1, x2, y1, y2 = area
    ny, nx = shape
    dx = float(x2 - x1) / float(nx - 1)
    dy = float(y2 - y1) / float(ny - 1)
    return [dy, dx]

def interp(x, y, v, shape, area=None, algorithm='cubic', extrapolate=False):
    """
    Interpolate data onto a regular grid.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * shape : tuple = (ny, nx)
        Shape of the interpolated regular grid, ie (ny, nx).
    * area : tuple = (x1, x2, y1, y2)
        The are where the data will be interpolated. If None, then will get the
        area from *x* and *y*.
    * algorithm : string
        Interpolation algorithm. Either ``'cubic'``, ``'nearest'``,
        ``'linear'`` (see scipy.interpolate.griddata), or ``'nn'`` for nearest
        neighbors (using matplotlib.mlab.griddata)
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * ``[x, y, v]``
        Three 1D arrays with the interpolated x, y, and v

    """
    if algorithm not in ['cubic', 'linear', 'nearest', 'nn']:
        raise ValueError("Invalid interpolation algorithm: " + str(algorithm))
    ny, nx = shape
    if area is None:
        area = (x.min(), x.max(), y.min(), y.max())
    x1, x2, y1, y2 = area
    xs = numpy.linspace(x1, x2, nx)
    ys = numpy.linspace(y1, y2, ny)
    xp, yp = [i.ravel() for i in numpy.meshgrid(xs, ys)]
    if algorithm == 'nn':
        grid = matplotlib.mlab.griddata(x, y, v, numpy.reshape(xp, shape),
                                        numpy.reshape(yp, shape),
                                        interp='nn').ravel()
        if extrapolate and numpy.ma.is_masked(grid):
            grid = extrapolate_nans(xp, yp, grid)
    else:
        grid = interp_at(x, y, v, xp, yp, algorithm=algorithm,
                         extrapolate=extrapolate)
    return [xp, yp, grid]


def interp_at(x, y, v, xp, yp, algorithm='cubic', extrapolate=False):
    """
    Interpolate data onto the specified points.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * xp, yp : 1D arrays
        Points where the data values will be interpolated
    * algorithm : string
        Interpolation algorithm. Either ``'cubic'``, ``'nearest'``,
        ``'linear'`` (see scipy.interpolate.griddata)
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * v : 1D array
        1D array with the interpolated v values.

    """
    if algorithm not in ['cubic', 'linear', 'nearest']:
        raise ValueError("Invalid interpolation algorithm: " + str(algorithm))
    grid = scipy.interpolate.griddata((x, y), v, (xp, yp),
                                      method=algorithm).ravel()
    if extrapolate and algorithm != 'nearest' and numpy.any(numpy.isnan(grid)):
        grid = extrapolate_nans(xp, yp, grid)
    return grid

def profile(x, y, v, point1, point2, size, extrapolate=False):
    """
    Extract a data profile between 2 points.

    Uses interpolation to calculate the data values at the profile points.

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.
    * point1, point2 : lists = [x, y]
        Lists the x, y coordinates of the 2 points between which the profile
        will be extracted.
    * size : int
        Number of points along the profile.
    * extrapolate : True or False
        If True, will extrapolate values outside of the convex hull of the data
        points.

    Returns:

    * [xp, yp, distances, vp] : 1d arrays
        ``xp`` and ``yp`` are the x, y coordinates of the points along the
        profile.
        ``distances`` are the distances of the profile points to ``point1``
        ``vp`` are the data points along the profile.

    """
    x1, y1 = point1
    x2, y2 = point2
    maxdist = numpy.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    distances = numpy.linspace(0, maxdist, size)
    angle = numpy.arctan2(y2 - y1, x2 - x1)
    xp = x1 + distances * numpy.cos(angle)
    yp = y1 + distances * numpy.sin(angle)
    vp = interp_at(x, y, v, xp, yp, algorithm='cubic', extrapolate=extrapolate)
    return xp, yp, distances, vp


def extrapolate_nans(x, y, v):
    """"
    Extrapolate the NaNs or masked values in a grid INPLACE using nearest
    value.

    .. warning:: Replaces the NaN or masked values of the original array!

    Parameters:

    * x, y : 1D arrays
        Arrays with the x and y coordinates of the data points.
    * v : 1D array
        Array with the scalar value assigned to the data points.

    Returns:

    * v : 1D array
        The array with NaNs or masked values extrapolated.

    """
    if numpy.ma.is_masked(v):
        nans = v.mask
    else:
        nans = numpy.isnan(v)
    notnans = numpy.logical_not(nans)
    v[nans] = scipy.interpolate.griddata((x[notnans], y[notnans]), v[notnans],
                                         (x[nans], y[nans]),
                                         method='nearest').ravel()
    return v


def cut(x, y, scalars, area):
    """
    Return a subsection of a grid.

    The returned subsection is not a copy! In technical terms, returns a slice
    of the numpy arrays. So changes made to the subsection reflect on the
    original grid. Use numpy.copy to make copies of the subsections and avoid
    this.

    Parameters:

    * x, y
        Arrays with the x and y coordinates of the data points.
    * scalars
        List of arrays with the scalar values assigned to the grid points.
    * area
        ``(x1, x2, y1, y2)``: Borders of the subsection

    Returns:

    * ``[subx, suby, subscalars]``
        Arrays with x and y coordinates and scalar values of the subsection.

    """
    xmin, xmax, ymin, ymax = area
    if len(x) != len(y):
        raise ValueError("x and y must have the same length")
    inside = [i for i in xrange(len(x))
              if x[i] >= xmin and x[i] <= xmax
              and y[i] >= ymin and y[i] <= ymax]
    return [x[inside], y[inside], [s[inside] for s in scalars]]

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
from shapely import geometry
import matplotlib.pyplot as plt

gdb='/home/ggrl/geodatabase/'

%matplotlib widget

from sources import source_code as f

In [3]:
dic_cartas, dic_raw_meta = f.get_region(escala='100k',id='SC23_Z_A_IV',geof='gama_1089',camada='litologia_100k')

importando dados geofisicos
# --- Iniciando seleção de área de estudo


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 folha cartográfica selecionada

['region', 'region_proj', 'raw_data', 'splines', 'scores', 'lito_splines', 'mean_score', 'litologia', 'cubic', 'lito_cubic']
Listando atributos dos dados geofisicos
# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']
lista de atributos geograficos = ['LONGITUDE', 'LATITUDE']
lista de atributos projetados = ['X', 'Y']

# --- Início da iteração entre as folhas cartográficas #


0it [00:00, ?it/s]

# Folha de código: SC23_Z_A_IV
 Atualizando dados brutos em dic_cartas['raw_data']
 com 81859 pontos de amostragem
epsg:32723


1it [01:24, 84.87s/it]

 Atualizando dados geologicos em dic_cartas['litologia']
 com 169 poligonos descritos por                         27 atributos geologicos 


In [4]:
dic_cartas['litologia']

{'SC23_Z_A_IV':        ID_UNIDADE    SIGLA    HIERARQUIA  \
 2816         1006     N1dl  Não definida   
 2820         1006     N1dl  Não definida   
 2821         4873   NP1rp3       Unidade   
 2823         4876  NP1rp2q    Litofácies   
 2825         5773   PP2cmv       Unidade   
 ...           ...      ...           ...   
 54755        4873   NP1rp3       Unidade   
 54756        1006     N1dl  Não definida   
 54761        1006     N1dl  Não definida   
 54771        4873   NP1rp3       Unidade   
 54773        1006     N1dl  Não definida   
 
                                                     NOME  \
 2816         Coberturas detrito-lateríticas ferruginosas   
 2820         Coberturas detrito-lateríticas ferruginosas   
 2821                            Unidade Rio Preto, xisto   
 2823   Litofácies Rio Preto, metassedimentos quartzos...   
 2825           Unidade Cristalândia do Piauí, metamáfica   
 ...                                                  ...   
 54755          

In [53]:
geof = dic_cartas['raw_data']['SC23_Z_A_IV']
area = dic_cartas['region_proj']['SC23_Z_A_IV']

litologia = dic_cartas['litologia']['SC23_Z_A_IV']

geof['geometry'] = [geometry.Point(x, y) for x, y in zip(geof['X'], geof['Y'])]

crs = "+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
gdf_geof = gpd.GeoDataFrame(geof, geometry='geometry', crs=crs)

In [54]:
area

(500000.0, 554712.5614995236, 8783975.089117344, 8839304.90587394)

In [4]:
geof = dic_cartas['raw_data']['SC23_Z_A_IV']


geof['geometry'] = [geometry.Point(x, y) for x, y in zip(geof['X'], geof['Y'])]

crs = "+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
gdf_geof = gpd.GeoDataFrame(geof, geometry='geometry', crs=crs)

# creating a grid with cells
xu, yu = regular(shape = (500, 500),
                     area = area)

sx, sy = spacing(shape = (500, 500),
                     area = area)

NameError: name 'area' is not defined

In [55]:
#lista_atributos_geof = ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']

CTCOR = np.array(gdf_geof.CTCOR)

MDT = np.array(gdf_geof.MDT)

eth = np.array(gdf_geof.eth)

eU = np.array(gdf_geof.eU)

KPERC = np.array(gdf_geof.KPERC)

THKRAZAO = np.array(gdf_geof.THKRAZAO)

UKRAZAO = np.array(gdf_geof.UKRAZAO)

UTHRAZAO = np.array(gdf_geof.UTHRAZAO)


x2, y2 = np.array(gdf_geof.X), np.array(gdf_geof.Y)


eth_ = interp_at(x2, y2, eth, xu, yu, algorithm = 'cubic', extrapolate = True)
eu_ = interp_at(x2, y2, eU, xu, yu, algorithm = 'cubic', extrapolate = True)
kperc_ = interp_at(x2, y2, KPERC, xu, yu, algorithm = 'cubic', extrapolate = True)
ctcor_ = interp_at(x2, y2, CTCOR, xu, yu, algorithm = 'cubic', extrapolate = True)
mdt_ = interp_at(x2, y2, MDT, xu, yu, algorithm = 'cubic', extrapolate = True)
uthrazao_ = interp_at(x2, y2, UTHRAZAO, xu, yu, algorithm = 'cubic', extrapolate = True)


# intialise data of lists. 
data = {'X':xu, 'Y':yu, 'MDT': mdt_,  'CTCOR': ctcor_,
        'KPERC': kperc_, 'eU':eu_, 'eth': eth_,
        'UTHRAZAO':uthrazao_} 
  
# Create DataFrame 
geof_1089 = pd.DataFrame(data)
  
# Print the output. 
geof_1089

,X,Y,MDT,CTCOR,KPERC,eU,eth,UTHRAZAO
0,500000.000000,8.783975e+06,510.971924,1728.869274,1.532959,2.267979,17.205152,0.133299
1,500109.644412,8.783975e+06,510.971924,1728.869274,1.532959,2.267979,17.205152,0.133299
2,500219.288824,8.783975e+06,510.971924,1728.869274,1.532959,2.267979,17.205152,0.133299
3,500328.933235,8.783975e+06,512.638234,1683.430977,1.503587,2.326873,16.427387,0.143720
4,500438.577647,8.783975e+06,513.651634,1640.614269,1.471212,2.356847,15.776541,0.151033
...,...,...,...,...,...,...,...,...
249995,554273.983852,8.839305e+06,477.906303,895.879587,0.651783,0.874006,10.362863,0.089454
249996,554383.628264,8.839305e+06,469.745255,862.128866,0.630352,0.961251,9.680775,0.106218
249997,554493.272676,8.839305e+06,460.988773,824.988883,0.605418,1.064014,8.921947,0.125131
249998,554602.917088,8.839305e+06,451.475640,783.606527,0.571166,1.182887,8.099838,0.145881


In [51]:
list_canal = dic_raw_meta['Lista_at_geof']
print(f"Lista de atributos: {list_canal}")

# LISTA DE TITULOS 
titles =   [
            'MDT (m)',
            'K (%)',
            'U (ppm)',
            'Th (ppm)',
            'Contagem Total (μR/h)',
            'Th (ppm) / K (%)',
            'U (ppm) / K (%)',
            'U (ppm) / Th (ppm)'
            ]


# DICIONARIO DE TITULOS
dic_titles = {}
for f, t in zip(list_canal, titles):
    dic_titles[f] = t

dic_titles

Lista de atributos: ['MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'THKRAZAO', 'UKRAZAO', 'UTHRAZAO']


{'MDT': 'MDT (m)',
 'KPERC': 'K (%)',
 'eU': 'U (ppm)',
 'eth': 'Th (ppm)',
 'CTCOR': 'Contagem Total (μR/h)',
 'THKRAZAO': 'Th (ppm) / K (%)',
 'UKRAZAO': 'U (ppm) / K (%)',
 'UTHRAZAO': 'U (ppm) / Th (ppm)'}

In [29]:
def plot_raw_data(raw_data):
    fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize = (12, 12),sharex='all',sharey='all')
    
    raw_data_describe = raw_data.describe(percentiles=(0.01,0.1,0.25,0.5,0.75,0.995))

    X, Y = raw_data.X, raw_data.Y

    for ax, f in zip(axs.flat, dic_titles):
        vmin = raw_data_describe[f]['1%']
        vmax = raw_data_describe[f]['99.5%']

        g = ax.scatter(c=raw_data[f], x = X,
                                      y = Y,
                                      cmap = 'rainbow',
                                      s    = 2.5,
                                      vmax =vmax, vmin=vmin)

        fig.colorbar(g, ax = ax)
        ax.set_title(str(dic_titles[f]), size = 11)
        ax.axis('scaled')

    #plt.tight_layout()

In [52]:
plot_raw_data(geof_1089)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'THKRAZAO'

In [22]:
geof_1089['geometry'] = [geometry.Point(x,y) for x, y in zip(geof_1089['X'], geof_1089['Y'])]

In [12]:
gdf = gpd.GeoDataFrame(geof_1089)

In [13]:
gdf = gdf.set_crs('EPSG:32723')

In [14]:
gdf = gdf.set_geometry('geometry')

In [15]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
region = dic_raw_meta['Malha_cartografica'].region[0]

litologia = dic_raw_meta['Litologia']
litologia=litologia.cx[region[0]:region[1],region[2]:region[3]]

KeyError: 'Litologia'

In [ ]:
#litologia.set_crs(32723,allow_override=True,inplace=True)
litologia.to_crs(32723,inplace=True)

In [ ]:
litologia.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
ax = litologia.plot('SIGLA')
plt.scatter(geof_1089.X, geof_1089.Y, cmap='hsv', s=5, c=geof_1089.eTh)
plt.colorbar(label=u'eTh')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(12, 8)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
type(geo_data)

pandas.core.frame.DataFrame

In [ ]:
geo_data

,X,Y,MDT,CT,KPERC,eU,eTh,geometry
0,448245.160000,7569035.34,1259.365337,1162.641011,0.645641,1.014426,12.998625,POINT (448245.16 7569035.34)
1,448348.902084,7569035.34,1293.523919,1014.894423,0.414994,1.392058,11.634912,POINT (448348.9020841683 7569035.34)
2,448452.644168,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448452.6441683367 7569035.34)
3,448556.386253,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448556.3862525051 7569035.34)
4,448660.128337,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448660.1283366734 7569035.34)
...,...,...,...,...,...,...,...,...
249995,499597.491663,7622519.96,965.094240,1513.728316,0.508648,0.947921,18.571917,POINT (499597.4916633398 7622519.96)
249996,499701.233748,7622519.96,976.609860,1648.207359,0.486749,1.300272,19.956881,POINT (499701.2337475082 7622519.96)
249997,499804.975832,7622519.96,991.446872,1804.825881,0.456061,1.690462,21.623599,POINT (499804.9758316766 7622519.96)
249998,499908.717916,7622519.96,991.446872,1804.825881,0.456061,1.690462,21.623599,POINT (499908.7179158449 7622519.96)


In [ ]:
SF_23['SIGLA'].iloc[SF_23.distance().idxmin()]

NameError: name 'SF_23' is not defined

In [ ]:
geo_data['closest_unid'] = gdf['geometry'].apply(lambda x: SF_23['SIGLA'].iloc[SF_23.distance(x).idxmin()])

IndexError: single positional indexer is out-of-bounds

In [ ]:
geo_data

,X,Y,MDT,CT,KPERC,eU,eTh,geometry
0,448245.160000,7569035.34,1259.365337,1162.641011,0.645641,1.014426,12.998625,POINT (448245.16 7569035.34)
1,448348.902084,7569035.34,1293.523919,1014.894423,0.414994,1.392058,11.634912,POINT (448348.9020841683 7569035.34)
2,448452.644168,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448452.6441683367 7569035.34)
3,448556.386253,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448556.3862525051 7569035.34)
4,448660.128337,7569035.34,1319.748234,736.232109,0.270898,0.629410,9.537711,POINT (448660.1283366734 7569035.34)
...,...,...,...,...,...,...,...,...
249995,499597.491663,7622519.96,965.094240,1513.728316,0.508648,0.947921,18.571917,POINT (499597.4916633398 7622519.96)
249996,499701.233748,7622519.96,976.609860,1648.207359,0.486749,1.300272,19.956881,POINT (499701.2337475082 7622519.96)
249997,499804.975832,7622519.96,991.446872,1804.825881,0.456061,1.690462,21.623599,POINT (499804.9758316766 7622519.96)
249998,499908.717916,7622519.96,991.446872,1804.825881,0.456061,1.690462,21.623599,POINT (499908.7179158449 7622519.96)
